In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing #Convert categorical data to neumeric values
from sklearn.neural_network import MLPClassifier #Model being used

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
keep_columns=['lat','month','contest-tmp2m-14d__tmp2m',
              'climateregions__climateregion']

In [3]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [4]:
def eliminate_columns(df):
    for col in df.columns:
        if col in keep_columns:
            print(col)
        else: 
            df=df.drop(col, axis=1)
    return df

In [5]:
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [6]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)

In [7]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [8]:
print('training set')
train_df=seperate_monthly(train_df)
train_df=eliminate_columns(train_df)
print('test set')
test_df=seperate_monthly(test_df)
test_df=eliminate_columns(test_df)

training set
lat
contest-tmp2m-14d__tmp2m
climateregions__climateregion
month
test set
lat
climateregions__climateregion
month


In [9]:
train_df

,lat,contest-tmp2m-14d__tmp2m,climateregions__climateregion,month
0,0.0,28.744480,0,9
1,0.0,28.370585,0,9
2,0.0,28.133059,0,9
3,0.0,28.256798,0,9
4,0.0,28.372353,0,9
...,...,...,...,...
375729,1.0,17.150954,9,8
375730,1.0,16.962051,9,8
375731,1.0,16.915474,9,8
375732,1.0,16.536761,9,8


In [10]:
y_train_df,X_train_df= seperate_y(train_df)

In [11]:
y_train_df=y_train_df.astype('int')

In [12]:
clf = MLPClassifier(hidden_layer_sizes=(2,10),
                    random_state=100,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf.fit(X_train_df,y_train_df)

Iteration 1, loss = 3.40323358
Iteration 2, loss = 3.08303248
Iteration 3, loss = 2.78333346
Iteration 4, loss = 2.76002665
Iteration 5, loss = 2.75446410
Iteration 6, loss = 2.75055503
Iteration 7, loss = 2.75061419
Iteration 8, loss = 2.74840022
Iteration 9, loss = 2.74691717
Iteration 10, loss = 2.74515032
Iteration 11, loss = 2.74255324
Iteration 12, loss = 2.74060331
Iteration 13, loss = 2.73913645
Iteration 14, loss = 2.73845771
Iteration 15, loss = 2.73705732
Iteration 16, loss = 2.73564482
Iteration 17, loss = 2.73595621
Iteration 18, loss = 2.73578437
Iteration 19, loss = 2.73548534
Iteration 20, loss = 2.73494189
Iteration 21, loss = 2.73427452
Iteration 22, loss = 2.73524559
Iteration 23, loss = 2.73475280
Iteration 24, loss = 2.73420767
Iteration 25, loss = 2.73351069
Iteration 26, loss = 2.73392183
Iteration 27, loss = 2.73271896
Iteration 28, loss = 2.73210968
Iteration 29, loss = 2.73236539
Iteration 30, loss = 2.73213020
Iteration 31, loss = 2.73225223
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(2, 10), learning_rate_init=0.01,
              random_state=100, verbose=True)

In [13]:
ypred=clf.predict(test_df)

In [14]:
#Trying to create a final 
final=pd.DataFrame()
final['contest-tmp2m-14d__tmp2m']=ypred
final['index']=0
i=len(train_df) #Legnth of train_data.csv
while i<len(train_df)+len(test_df):
    final['index'][i-len(train_df)]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,20,352604
1,20,352605
2,20,352606
3,20,352607
4,20,352608
...,...,...
31349,0,383953
31350,0,383954
31351,0,383955
31352,0,383956


In [15]:
final.to_csv('final.csv',index=False)

In [16]:
final

,contest-tmp2m-14d__tmp2m,index
0,20,352604
1,20,352605
2,20,352606
3,20,352607
4,20,352608
...,...,...
31349,0,383953
31350,0,383954
31351,0,383955
31352,0,383956
